In [4]:
from collections import Counter
import math

In [3]:
1

corpus_of_documents = [
"Take a leisurely walk in the park and enjoy the fresh air.",
"Visit a local museum and discover something new.",
"Attend a live music concert and feel the rhythm.",
"Go for a hike and admire the natural scenery.",
"Have a picnic with friends and share some laughs.",
"Explore a new cuisine by dining at an ethnic restaurant.",
"Take a yoga class and stretch your body and mind.",
"Join a local sports league and enjoy some friendly competition.",
"Attend a workshop or lecture on a topic you're interested in.",
"Visit an amusement park and ride the roller coasters."
]

Cosine similarity

In [1]:
user_query = "I am an AI engineer and i love learning new things in this filed"

In [2]:
document = "AI is a great and awesome filed for student who wants to learn about technology"

In [9]:
query_tokens = user_query.lower().split(" ")

In [10]:
document_tokens = document.lower().split(" ")

In [11]:
query_tokens, document_tokens

(['i',
  'am',
  'an',
  'ai',
  'engineer',
  'and',
  'i',
  'love',
  'learning',
  'new',
  'things',
  'in',
  'this',
  'filed'],
 ['ai',
  'is',
  'a',
  'great',
  'and',
  'awesome',
  'filed',
  'for',
  'student',
  'who',
  'wants',
  'to',
  'learn',
  'about',
  'technology'])

In [21]:
query_counter = Counter(query_tokens)
query_counter

Counter({'i': 2,
         'am': 1,
         'an': 1,
         'ai': 1,
         'engineer': 1,
         'and': 1,
         'love': 1,
         'learning': 1,
         'new': 1,
         'things': 1,
         'in': 1,
         'this': 1,
         'filed': 1})

In [15]:
documents_counter = Counter(document_tokens)

In [23]:
lst = []

for token in query_counter.keys():
    lst.append(query_counter[token])

#Sentence vector
lst

[2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [43]:
lst_2 = []
for token in documents_counter.keys():
    lst_2.append(documents_counter[token])

lst_2

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [36]:
same_token_list = []

for tokens in query_counter.keys() & documents_counter.keys():
    print(tokens)
    same_token_list.append(query_counter[tokens] * documents_counter[tokens])

ai
filed
and


Dot product of the vectors

In [44]:
dot_product = sum(same_token_list)

Magnitude of the vectors

In [39]:
query_magnitude = math.sqrt(sum(query_counter[token] ** 2 for token in query_counter))

In [40]:
query_magnitude

4.0

In [42]:
document_magnitude = math.sqrt(sum(documents_counter[tokens] ** 2 for tokens in documents_counter))

document_magnitude

3.872983346207417

final Cosine similarity

In [45]:
final_cosin_similarity = dot_product / (query_magnitude * document_magnitude)

final_cosin_similarity

0.19364916731037085

In [47]:
def cosine_similarity(query, document):
    # Tokenize and convert to lowercase
    query_token = query.lower().split(" ")
    document_token = document.lower().split(" ")

    # Create counters for query and documents
    query_counter = Counter(query_token)
    document_counter = Counter(document_token)

    # Calculate the dot product
    dot_product = sum(query_counter[token] * document_counter[token] for token in query_counter)

    # Calculate magnitude 
    query_magnitude = math.sqrt(sum(query_counter[token] ** 2 for token in query_counter))
    document_magnitude = math.sqrt(sum(document_counter[token] ** 2 for token in document_counter))

    # Cosine similarity
    cosine_similarity = dot_product / (query_magnitude * document_magnitude)

    return cosine_similarity

In [48]:
user_query_1 = "Is playing video games healthy for body?"
document_1 = "Playing video games for a long time during the day can be really harmful for body"


In [49]:
cosine_similarity(user_query_1, document_1)

0.44543540318737396

In [51]:
def return_response(query, corpus):
    similarites = []
    for doc in corpus:
        similarity = cosine_similarity(query, doc)
        similarites.append(similarity)

    return corpus_of_documents[similarites.index(max(similarites))]

In [53]:
corpus_of_documents

['Take a leisurely walk in the park and enjoy the fresh air.',
 'Visit a local museum and discover something new.',
 'Attend a live music concert and feel the rhythm.',
 'Go for a hike and admire the natural scenery.',
 'Have a picnic with friends and share some laughs.',
 'Explore a new cuisine by dining at an ethnic restaurant.',
 'Take a yoga class and stretch your body and mind.',
 'Join a local sports league and enjoy some friendly competition.',
 "Attend a workshop or lecture on a topic you're interested in.",
 'Visit an amusement park and ride the roller coasters.']

In [56]:
user_input = "how can i enjoy sport?"

In [58]:
relevant_document = return_response(user_input, corpus_of_documents)

Augment the response by using LLAMA2 model

In [ ]:
import json
import requests

full_response = []

prompt = """

You are a bot that makes reommendations for activities. You answer in very short sentences and do not include extra information.
This is the recommended activity: {relevant_document}
The user input is: {user_input}
compile a recommendation to the user bases on the recommended activity and the user input

"""

url = "https://localhost:11434/api/generate"
data = {
    "model":"llama2",
    "prompt": prompt.format(user_input=user_input, relevant_doocument=relevant_document)

}

headers = {"Content-Type": "application/json"}
response = requests.post(url, data=json.dumps(data), headers=headers, stream=True)
try:
    for line in response.iter_lines():
        # filter out keep-alive new lines
        if line:
            decoded_line = json.loads(line.decode("utf-8"))
            # print(decoded_line["response"]) # Uncomment to results, token by token
            full_response.append(decoded_line["response"])

finally:
    response.close()

print("".join(full_response))